# Variational Inference

When there is no conjugacy between prior and posterior, we need to make approximation to the posterior as it can't be derived in closed form. Variational Inference refers to the approximation $q(\theta) \approx p(\theta|x)$ such that the Kullback-Leibler (KL) divergence is minimized, i.e.

\begin{equation*}

q^*(\theta) = \argmin_{q(\theta)} KL(q(\theta)||p(\theta|x)) 
\end{equation*}


where the KL divergence is defined as 
\begin{equation*}
KL(q(\theta)||p(\theta|x)) = \int q(\theta) \log \frac{q(\theta)}{p(\theta|x)} d\theta
\end{equation*}

A first glance to the equation is that if $q(\theta) = p(\theta|x)$ then the KL divergence equals zero. Under the same domain, it is a good measure of the difference between $q$ and $p$. Note also that 

1. $KL(q||p) \neq KL(p||q)$
2. $KL(q||p) \ge 0$

The KL divergence depends on the posterior which is what we want to calculate in the first place. Luckily we have the following math trick:
\begin{align*}
KL(q(\theta)||p(\theta|x)) &= \int q(\theta) \log \frac{q(\theta)}{p(\theta|x)} d\theta\\
&= \int q(\theta)\log q(\theta)d\theta - \int q(\theta) \log p(\theta|x) d\theta\\
&= \mathbb{E}_{q(\theta)} \log q(\theta) - \mathbb{E}_{q(\theta)} \log \frac{p(\theta,x)}{p(x)}\\
&= \mathbb{E}_{q(\theta)} \log q(\theta) - \mathbb{E}_{q(\theta)} \log p(\theta,x) + \mathbb{E}_{q(\theta)}\log p(x)\\
&=\mathbb{E}_{q(\theta)} \log q(\theta) - \mathbb{E}_{q(\theta)} \log p(\theta,x) + \log p(x)\\
&= -L(q(\theta)) + \log p(x) 
\end{align*}



On the RHS, we can compute the joint probability $p(\theta,x)$ and we do not care about $p(x)$ as it does not depend on $q(\theta)$. Therefore, minimizing $KL(q||p)$ is equivalent to maximizing $L(q)$. The optimization problem becomes:
\begin{equation*}
    q^*(\theta) = \argmax_{q(\theta)} \mathbb{E}_{q(\theta)} \log \frac{p(\theta, x)}{q(\theta)}
\end{equation*}


The term $L(q)$ is called Evidence Lower Bound (ELBO) as $KL(q||p) \ge 0$ and thus $\log p(x)\ge L(q)$.

### Mean field approximation

In general, $\theta$ refers to all varaibles, including latent variables (e.g. gaussian mixture model). It would be difficult to optimize with repect to all variables at the same time. Similar to physics problem, mean field approximation allows us to neglect correlation between variables. Let m be the number of variables, then 
\begin{equation*}
q(\theta) = \prod_{j=1}^m q_j(\theta_j)
\end{equation*}
Obviously the variables are independent of each other now. The ELBO becomes
\begin{equation*}
L(q(\theta)) = \mathbb{E}_{q(\theta)} \log p(\theta,x) - \mathbb{E}_{q(\theta)} \sum_{j=1}^m\log q_j(\theta_j)
\end{equation*}


#### Block coordinate assent
Same idea as greedy algorithm. At each step, fix all other variables $\theta_j\neq\theta_k$ and optimize with respect to $\theta_k$. For $\theta_k$, the ELBO reduces to 
\begin{align*}
L(q(\theta)) & =\mathbb{E}_{q(\theta)} \log p(\theta,x) - \mathbb{E}_{q_k(\theta_k)} \log q_k(\theta_k) + \text{const}
\end{align*}

Using the chain rule $p(\theta,x) = p(\theta_k|\theta_{-k},x) p(\theta_{-k}|x) p(x)$, we have
\begin{align*}
L(q(\theta)) & =\mathbb{E}_{q(\theta)} \bigg[\log p(\theta_k|\theta_{-k},x)  + \log p(\theta_{-k}|x) + \log p(x)\bigg] - \mathbb{E}_{q_k(\theta_k)} \log q_k(\theta_k) + \text{const}\\
&= \mathbb{E}_{q(\theta)}\log p(\theta_k|\theta_{-k},x) + \mathbb{E}_{q(\theta)}\log p(\theta_{-k}|x) + \log p(x)- \mathbb{E}_{q_k(\theta_k)} \log q_k(\theta_k) + \text{const}\\
&= \mathbb{E}_{q_k(\theta_k)}\mathbb{E}_{q_{-k}(\theta_{-k})}\log p(\theta_k|\theta_{-k},x)- \mathbb{E}_{q_k(\theta_k)} \log q_k(\theta_k) + \text{const}\\
\end{align*}

Each optimization for $\theta_k$ is subject to the constraint
\begin{equation*}
\int q_k(\theta_k) d\theta_k = 1
\end{equation*}

Applying the Lagrange multiplier, we have 
\begin{equation*}
    F(q_k) = \int q_k(\theta_k)\mathbb{E}_{q_{-k}(\theta_{-k})}\log p(\theta_k|\theta_{-k},x) d\theta_k - \int q_k(\theta_k) \log q_k(\theta_k) d\theta_k - \lambda \bigg(\int q_k(\theta_k)d \theta_k -1 \bigg)
\end{equation*}

Varaiating with respect to $q_k$ and $\lambda$,
\begin{gather*}
\frac{\delta F}{\delta q_k} = \mathbb{E}_{q_{-k}(\theta_{-k})}\log p(\theta_k|\theta_{-k},x) - \log q_k(\theta_k) + 1- \lambda = 0 \\ 
\frac{\delta F}{\delta \lambda} = \int q_k(\theta_k)d\theta_k - 1 = 0
\end{gather*}


The first equation gives:
\begin{equation*}
q_k(\theta_k) = \exp\bigg[\mathbb{E}_{q_{-k}(\theta_{-k})}\log p(\theta_k|\theta_{-k},x)\bigg] \exp(1-\lambda)
\end{equation*}

The second equation just states the normalization, i.e.
\begin{equation*}
q_k(\theta_k) = \frac{\exp\bigg[\mathbb{E}_{q_{-k}(\theta_{-k})}\log p(\theta_k|\theta_{-k},x)\bigg]}{\int \exp\bigg[\mathbb{E}_{q_{-k}(\theta_{-k})}\log p(\theta_k|\theta_{-k},x)\bigg]d\theta_k}
\end{equation*}

Since the conditional probability can be written as $p(\theta_k|\theta_{-k},x) = p(\theta, x) / p(\theta_{-k},x)$, the above expression can be rewritten as 
\begin{equation*}
q_k(\theta_k) = \frac{\exp\bigg[\mathbb{E}_{q_{-k}(\theta_{-k})}\log p(\theta,x) \bigg]}{\int \exp\bigg[\mathbb{E}_{q_{-k}(\theta_{-k})}\log p(\theta,x)  \bigg]d\theta_k}
\end{equation*}

### Algorithm : Coordinate Ascent Pseudo-Code for Variational Inference

Input : data, model $p(x,\theta)$ 

Output : $q(\theta)$

1. Initialize $q(\theta) = \prod_{k=1}^m q_k(\theta_k)$
2. while not convergence, update $q_k(\theta_k) \propto \exp\bigg[\mathbb{E}_{q_{-k}(\theta_{-k})}\log p(\theta,x) \bigg] $ for each k. 
3. calculate ELBO
4. repeat until convergence.

Note that in step 2 the update depends on the fact that we could calculate $\mathbb{E}_{q_{-k}(\theta_{-k})}\log p(\theta,x)$ for those fixed varaiables. That is to say, this algorithm is valid only if the prior and posterior satisfy conditional conjugacy, i.e. there is conjugacy for each \theta_k while fixing other \theta.